# lab 10d

## Notes

**[Gist link](https://gist.github.com/ThatNerdSquared/e9a8acce736474536cd2f0db5cd0ba55)**

As far as I can tell, we really need to find two things:
- Trial Ion Product
    - To find this, we need to find the molarity of each element
    - To find molarity of each element, we need to find molarity of the compound
    - To find molarity of the compound, we need to find mols of compound
    - To find mols of compound, we need to convert grams to mols via `grams over molar mass`
    - So the pipeline is basically:
        - Take grams, convert to mols using $\frac{\text{1 mol}}{\text{(molar mass of X)g}}$, convert to molarity by dividing that by volume in L (convert that from mL), find molarity of each element by looking at the ratios, and then calculate reaction quotient using the proper exponents
- Solubility Product Constant
    - This is experimentally determined, isn't it?
    - We could use the grams per L solubility from PubChem...
        - essentially follow [this method](https://www.chemteam.info/Equilibrium/Calc-Ksp-FromMolSolub.html)
        - convert it to mols per L (molar solubility)
        - calculate $K_{sp}$ via reaction quotient formula
        - We [can't grab solubility programmatically](https://github.com/mcs07/PubChemPy/issues/16) though so it could be a pain
- Possibly useful libs:
    - https://pypi.org/project/chemparse/
    - https://github.com/cgohlke/molmass
    - https://pypi.org/project/pyvalem/
    - automated chemical equation balancing: https://www.wikiwand.com/en/Chemical_equation#System_of_linear_equations, https://pythonnumericalmethods.berkeley.edu/notebooks/chapter14.05-Solve-Systems-of-Linear-Equations-in-Python.html

In [5]:
# Import necessary packages.
import pandas as pd
import chemparse

In [6]:
# Helper func for styling dataframes.
def print_styled_df(df, caption):
    df = df.style.set_caption(caption).format(precision=2)
    display(df)

In [7]:
# Reading in external data.
solutionset_1 = pd.read_csv('solutionset-1.csv').set_index('Stock solutions')
solutionset_2 = pd.read_csv('solutionset-2.csv').set_index('Stock solutions')
solutionset_3 = pd.read_csv('solutionset-3.csv').set_index('Stock solutions')
pubchem_elem = pd.read_csv('pubchem-elements.csv').set_index('Symbol')

The $CuCl_2$ used in Solution Set I was anhydrous.

In [8]:
print_styled_df(solutionset_1, 'Solution Set I')

,Mass,Volume
Stock solutions,,
Cu(NO3)2,0.87,30.86
CuCl2,0.50,31.18
K2CO3,0.71,42.70
NaNO3,0.52,29.69
KCl,0.45,30.45


In [9]:
print_styled_df(solutionset_2, 'Solution Set II')

,Mass,Volume
Stock solutions,,
(Na2SO4)(10H2O),1.16,29.75
Sr(NO3)2,0.91,30.00
Ba(NO3)2,0.80,30.53
AlCl3,0.86,30.54
K_2SO4,1.11,30.60


In [10]:
print_styled_df(solutionset_3, 'Solution Set III')

,Mass,Volume
Stock solutions,,
Fe(NO3)3 * 9H2O,1.25,29.90
Co(NO3)2,1.12,30.48
CoCl2,0.70,30.24
NaOH,0.25,30.10
KOH,0.34,30.00
NaNO3,0.50,29.87


In [11]:
pubchem_elem

,AtomicNumber,Name,AtomicMass,CPKHexColor,ElectronConfiguration,Electronegativity,AtomicRadius,IonizationEnergy,ElectronAffinity,OxidationStates,StandardState,MeltingPoint,BoilingPoint,Density,GroupBlock,YearDiscovered
Symbol,,,,,,,,,,,,,,,,
H,1,Hydrogen,1.008000,FFFFFF,1s1,2.20,120.0,13.598,0.754,"+1, -1",Gas,13.81,20.28,0.000090,Nonmetal,1766
He,2,Helium,4.002600,D9FFFF,1s2,NaN,140.0,24.587,NaN,0,Gas,0.95,4.22,0.000179,Noble gas,1868
Li,3,Lithium,7.000000,CC80FF,[He]2s1,0.98,182.0,5.392,0.618,+1,Solid,453.65,1615.00,0.534000,Alkali metal,1817
Be,4,Beryllium,9.012183,C2FF00,[He]2s2,1.57,153.0,9.323,NaN,+2,Solid,1560.00,2744.00,1.850000,Alkaline earth metal,1798
B,5,Boron,10.810000,FFB5B5,[He]2s2 2p1,2.04,192.0,8.298,0.277,+3,Solid,2348.00,4273.00,2.370000,Metalloid,1808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Fl,114,Flerovium,290.192000,NaN,[Rn]7s2 7p2 5f14 6d10 (predicted),NaN,NaN,NaN,NaN,"6, 4,2, 1, 0",Expected to be a Solid,NaN,NaN,NaN,Post-transition metal,1998
Mc,115,Moscovium,290.196000,NaN,[Rn]7s2 7p3 5f14 6d10 (predicted),NaN,NaN,NaN,NaN,"3, 1",Expected to be a Solid,NaN,NaN,NaN,Post-transition metal,2003
Lv,116,Livermorium,293.205000,NaN,[Rn]7s2 7p4 5f14 6d10 (predicted),NaN,NaN,NaN,NaN,"+4, +2, -2",Expected to be a Solid,NaN,NaN,NaN,Post-transition metal,2000


Let's try out calculating molarity of items from Solution Set I. We need to:

- find the molar mass of the compound

In [12]:
# Finding molar mass of a given chemical formula.
def formula_to_molar_mass(formula):
    parsed_formula = chemparse.parse_formula(formula)
    total_molar_mass = 0
    for item in parsed_formula.keys():
        amount = parsed_formula[item]
        mm = pubchem_elem['AtomicMass'][item]
        total_molar_mass += mm * amount
        print(item, amount, mm)
    return total_molar_mass

- find the number of mols of the compound in the solution
- find the number of mols per L

In [13]:
# Finding molarity from molar mass.
def molar_mass_to_molarity(symbol, molar_mass, solutionset_df):
    mols_count = solutionset_df['Mass'][symbol] / molar_mass
    molarity = mols_count / (solutionset_df['Volume'][symbol] / 1000)
    return molarity

These operations need to be performed over the whole set:

In [14]:
def solution_set_to_molarity(ss_df):
    molarity_set = {}

    for index, row in ss_df.iterrows():
        molar_mass = formula_to_molar_mass(index)
        molarity_set[index] = molar_mass_to_molarity(index, molar_mass, ss_df)

    ss_molarity_df = pd.DataFrame.from_dict(
        molarity_set,
        orient='index',
        columns=['Molarity (M)']
    )
    return ss_molarity_df

Let's look at our results:

In [15]:
print_styled_df(solution_set_to_molarity(solutionset_1), 'Molarity in Solution Set I')

N 2.0 14.007
Cu 1.0 63.55
O 6.0 15.999
Cu 1.0 63.55
Cl 2.0 35.45
K 2.0 39.0983
C 1.0 12.011
O 3.0 15.999
Na 1.0 22.9897693
N 1.0 14.007
O 3.0 15.999
K 1.0 39.0983
Cl 1.0 35.45


,Molarity (M)
Cu(NO3)2,0.15
CuCl2,0.12
K2CO3,0.12
NaNO3,0.21
KCl,0.20


In [12]:
print_styled_df(solution_set_to_molarity(solutionset_2), 'Molarity in Solution Set II')

,Molarity (M)
(Na2SO4)(10H2O),0.27
Sr(NO3)2,0.14
Ba(NO3)2,0.10
AlCl3,0.21
K_2SO4,0.93


In [14]:
print_styled_df(solution_set_to_molarity(solutionset_3), 'Molarity in Solution Set III')

,Molarity (M)
Fe(NO3)3 * 9H2O,0.17
Co(NO3)2,0.20
CoCl2,0.18
NaOH,0.21
KOH,0.20
NaNO3,0.20


## Calculating Trial Ion Products

Let's look at the double replacements for Set I:

In [46]:
set1_col = [
    'Cu(NO3)2',
    'CuCl2',
    'K2CO3',
    'NaNO3',
    'KCl'
]

set1_drs = {
    'Cu(NO3)2': ['N/A', ['Cu(NO3)2', 'CuCl2'], ['CuCO3', 'K2(NO3)2'], ['Cu(NO3)2', '2NaNO3'], ['CuCl2', '2KCl']],
    'CuCl2': ['Same', 'N/A', ['CuCO3', '2KCL'], ['Cu(NO3)2', '2NaCl'], ['CuCl2', '2KCl']],
    'K2CO3': ['Same', 'Same', 'N/A', ['2KNO3', 'Na2CO3'], ['K2CO3', '2KCl']],
    'NaNO3': ['Same', 'Same', 'Same', 'N/A', ['NaCl', 'KNO3']],
    'KCl': ['Same', 'Same', 'Same', 'Same', 'N/A']
}

ss1_matrix = pd.DataFrame.from_dict(set1_drs, orient='index', columns=set1_col)
print_styled_df(ss1_matrix, 'Solution Set I Trials')

,Cu(NO3)2,CuCl2,K2CO3,NaNO3,KCl
Cu(NO3)2,N/A,"['Cu(NO3)2', 'CuCl2']","['CuCO3', 'K2(NO3)2']","['Cu(NO3)2', '2NaNO3']","['CuCl2', '2KCl']"
CuCl2,Same,N/A,"['CuCO3', '2KCL']","['Cu(NO3)2', '2NaCl']","['CuCl2', '2KCl']"
K2CO3,Same,Same,N/A,"['2KNO3', 'Na2CO3']","['K2CO3', '2KCl']"
NaNO3,Same,Same,Same,N/A,"['NaCl', 'KNO3']"
KCl,Same,Same,Same,Same,N/A


For each product, we need to calculate the Trial Ion Product, and compare it to the Solubility Product Constant. To calculate trial ion products, we need to:
- Find the molarities of each item based on previous molarity calculations
- Calculate based on coefficients

In [ ]:
set1_drs = {
    'CuNO32': 
}

In [54]:
set2_col = [
    '(Na2SO4)(10H2O)',
    'Sr(NO3)2',
    'Ba(NO3)2',
    'AlCl3',
    'K_2SO4'
]

set2_drs = {
    '(Na2SO4)(10H2O)': ['Same'] * 5,
    'Sr(NO3)2': [None] * 5,
    'Ba(NO3)2': [None] * 5,
    'AlCl3': [None] * 5,
    'K_2SO4': [None] * 5
}

ss2_matrix = pd.DataFrame.from_dict(set2_drs, orient='index', columns=set2_col)
print_styled_df(ss2_matrix, 'Solution Set II Trials')

,(Na2SO4)(10H2O),Sr(NO3)2,Ba(NO3)2,AlCl3,K_2SO4
(Na2SO4)(10H2O),Same,Same,Same,Same,Same
Sr(NO3)2,None,None,None,None,None
Ba(NO3)2,None,None,None,None,None
AlCl3,None,None,None,None,None
K_2SO4,None,None,None,None,None


## Results
- tables of actual data we collected
- tables w calculations comparing reaction quotients to $K_{sp}$s
- summary of what was obtained, basically

## Discussion

- hypothesis => hootonian intervention required
- results significance => how the results relate back to core point of lab ($K_{sp}$s? solutbility?)
- scientific explanation => calculate $K_{sp}$s to explain why things played out the way they did
- error analysis => basically saying visual/contaimination/temp, and mostly saying they're negligible